<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Implementing_Differentially_Private_Training_with_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opacus

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine

# Define your model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)  # Example model

    def forward(self, x):
        return self.fc(x)

# Create some dummy data
data = torch.randn(1000, 10)
labels = torch.randint(0, 2, (1000, 1)).float()
dataset = TensorDataset(data, labels)

# Initialize model and data
model = MyModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the privacy engine
privacy_engine = PrivacyEngine()

# Attach privacy engine to the model and optimizer
model, optimizer, dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=dataloader,
    noise_multiplier=1.1,
    max_grad_norm=1.0
)

# Define the loss function
loss_fn = nn.BCEWithLogitsLoss()

# Training loop with differential privacy
for epoch in range(10):  # Example for 10 epochs
    total_loss = 0.0
    for batch_data, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/10], Loss: {avg_loss:.4f}")